In [ ]:
# Install Required Packages
# Run this cell first to install all necessary dependencies

import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")

# List of required packages
packages = [
    "torch>=1.9.0",
    "transformers>=4.20.0", 
    "huggingface_hub>=0.10.0",
    "tokenizers>=0.12.0",
    "datasets",
    "accelerate",
]

print("Installing required packages for Hugging Face model demonstration...")
print("This may take a few minutes...\n")

for package in packages:
    install_package(package)

print("\n" + "="*60)
print("📦 Package installation complete!")
print("💡 For GPU support, you may also want to install:")
print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
print("="*60)

In [ ]:
# Hugging Face Model Demonstration with non Generative AI Language Models
## Loading Models with Authentication and Performing NLP Tasks

#This notebook demonstrates:
#1. Setting up Hugging Face authentication
#2. Loading models for summarization and extractive QA
#3. Using simple and complex prompts
#4. Comparing different model outputs

In [ ]:
# Import required libraries
import os
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    AutoModelForQuestionAnswering,
    pipeline
)
from huggingface_hub import login
import torch
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Authentication Setup
# Option 1: Set your Hugging Face token as an environment variable
# os.environ["HUGGINGFACE_HUB_TOKEN"] = "your_token_here"

# Option 2: Use huggingface_hub login (recommended for interactive use)
# Uncomment the line below and run to authenticate interactively
# login()

# Option 3: Login programmatically (for automation)
# login(token="your_token_here")

# For this demo, we'll check if authentication is available
try:
    from huggingface_hub import HfApi
    api = HfApi()
    user_info = api.whoami()
    print(f"Authenticated as: {user_info['name']}")
    authenticated = True
except Exception as e:
    print("Not authenticated - will use public models only")
    print("To authenticate, uncomment one of the login options above")
    authenticated = False

In [ ]:
# Load Summarization Models
print("Loading summarization models...")

# Model 1: BART for summarization (good for general purpose)
summarizer_bart = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)

# Model 2: T5 for summarization (good for instruction-following)
summarizer_t5 = pipeline(
    "text2text-generation",
    model="t5-small",  # Use t5-base or t5-large for better quality
    tokenizer="t5-small",
    device=0 if torch.cuda.is_available() else -1
)

print("✓ Summarization models loaded successfully!")
print(f"BART model device: {summarizer_bart.device}")
print(f"T5 model device: {summarizer_t5.device}")

In [ ]:
# Load Question Answering Models
print("Loading question answering models...")

# Model 1: DistilBERT for QA (fast and efficient)
qa_distilbert = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    tokenizer="distilbert-base-cased-distilled-squad",
    device=0 if torch.cuda.is_available() else -1
)

# Model 2: RoBERTa for QA (higher accuracy)
qa_roberta = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    tokenizer="deepset/roberta-base-squad2",
    device=0 if torch.cuda.is_available() else -1
)

print("✓ Question answering models loaded successfully!")
print(f"DistilBERT QA device: {qa_distilbert.device}")
print(f"RoBERTa QA device: {qa_roberta.device}")

In [ ]:
# Sample Texts for Demonstration

# Simple text for basic testing
simple_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans. 
AI research has been highly successful in developing effective techniques for solving a wide range of problems, from game playing to medical diagnosis.
"""

# Complex text for advanced testing
complex_text = """
The field of artificial intelligence has undergone remarkable transformations since its inception in the 1950s. 
Initially focused on symbolic reasoning and expert systems, AI research has evolved to encompass machine learning, 
deep learning, and neural networks. Modern AI systems leverage vast amounts of data and computational power to 
perform tasks that were once thought to require human-level intelligence.

Large language models, such as GPT and BERT, have revolutionized natural language processing by demonstrating 
unprecedented capabilities in text generation, comprehension, and translation. These models are trained on 
massive datasets containing billions of words from diverse sources, enabling them to capture complex patterns 
in human language.

The applications of AI span numerous domains, including healthcare, finance, transportation, and entertainment. 
In healthcare, AI systems assist in medical imaging analysis, drug discovery, and personalized treatment plans. 
Autonomous vehicles rely on computer vision and sensor fusion to navigate complex traffic scenarios. 
Recommendation systems power content discovery on streaming platforms and e-commerce websites.

However, the rapid advancement of AI also raises important ethical considerations. Issues such as algorithmic bias, 
privacy concerns, job displacement, and the potential for misuse of AI technologies require careful attention 
from researchers, policymakers, and society as a whole. Ensuring that AI development proceeds in a responsible 
and beneficial manner remains one of the most significant challenges of our time.
"""

print("Sample texts loaded:")
print(f"Simple text length: {len(simple_text)} characters")
print(f"Complex text length: {len(complex_text)} characters")

## Summarization Tasks

### Simple Prompts
Let's start with basic summarization using different models and compare their outputs.

In [ ]:
# Simple Summarization with BART
print("=== Simple Summarization with BART ===")
print("Original text:")
print(simple_text)
print("\n" + "="*50 + "\n")

# Basic summarization
bart_summary = summarizer_bart(simple_text, max_length=50, min_length=20, do_sample=False)
print("BART Summary:")
print(bart_summary[0]['summary_text'])

print("\n" + "="*50 + "\n")

# T5 summarization (requires different prompt format)
t5_input = "summarize: " + simple_text
t5_summary = summarizer_t5(t5_input, max_length=60, min_length=20, do_sample=False)
print("T5 Summary:")
print(t5_summary[0]['generated_text'])

In [ ]:
# Complex Summarization with Different Styles
print("=== Complex Text Summarization ===")

# Standard summarization
print("1. Standard Summary (BART):")
bart_complex = summarizer_bart(complex_text, max_length=100, min_length=50, do_sample=False)
print(bart_complex[0]['summary_text'])

print("\n" + "-"*40 + "\n")

# Bullet point style summary with T5
print("2. Bullet Point Style Summary (T5):")
bullet_prompt = "summarize in bullet points: " + complex_text
t5_bullets = summarizer_t5(bullet_prompt, max_length=150, min_length=50, do_sample=False)
print(t5_bullets[0]['generated_text'])

print("\n" + "-"*40 + "\n")

# Executive summary style
print("3. Executive Summary Style (T5):")
exec_prompt = "write an executive summary: " + complex_text
t5_exec = summarizer_t5(exec_prompt, max_length=120, min_length=60, do_sample=False)
print(t5_exec[0]['generated_text'])

print("\n" + "-"*40 + "\n")

# Short social media style summary
print("4. Social Media Style Summary (BART - very short):")
bart_short = summarizer_bart(complex_text, max_length=30, min_length=15, do_sample=False)
print(bart_short[0]['summary_text'])

## Extractive Question Answering Tasks

### Simple Questions
Let's test the QA models with straightforward questions.

In [ ]:
# Simple Question Answering
def ask_question(question, context, model_name, qa_pipeline):
    """Helper function to ask questions and display results"""
    result = qa_pipeline(question=question, context=context)
    print(f"Model: {model_name}")
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['score']:.4f}")
    print(f"Context span: ...{context[result['start']:result['end']]}...")
    print("-" * 50)

# Simple questions on simple text
simple_questions = [
    "What is artificial intelligence?",
    "What has AI research been successful in?",
    "What type of intelligence do humans display?"
]

print("=== Simple QA on Simple Text ===\n")

for question in simple_questions:
    ask_question(question, simple_text, "DistilBERT", qa_distilbert)
    ask_question(question, simple_text, "RoBERTa", qa_roberta)
    print("="*60)

In [ ]:
# Complex Question Answering on Complex Text
complex_questions = [
    "When was the field of artificial intelligence founded?",
    "What are some examples of large language models mentioned?",
    "In which domains are AI applications mentioned?",
    "What ethical considerations are raised about AI?",
    "What techniques were AI initially focused on?",
    "How do autonomous vehicles use AI?",
    "What is required for responsible AI development?"
]

print("=== Complex QA on Complex Text ===\n")

for i, question in enumerate(complex_questions, 1):
    print(f"Question {i}:")
    
    # Get answers from both models
    distilbert_result = qa_distilbert(question=question, context=complex_text)
    roberta_result = qa_roberta(question=question, context=complex_text)
    
    print(f"Q: {question}")
    print(f"DistilBERT: {distilbert_result['answer']} (confidence: {distilbert_result['score']:.3f})")
    print(f"RoBERTa: {roberta_result['answer']} (confidence: {roberta_result['score']:.3f})")
    
    # Highlight if there's a significant confidence difference
    conf_diff = abs(distilbert_result['score'] - roberta_result['score'])
    if conf_diff > 0.2:
        print(f"⚠️  Large confidence difference: {conf_diff:.3f}")
    
    print("-" * 80)

In [ ]:
# Advanced QA Techniques
print("=== Advanced QA Techniques ===\n")

# 1. Multi-hop reasoning (questions requiring combining information)
print("1. Multi-hop Reasoning:")
multi_hop_q = "What challenges exist for AI applications in healthcare and transportation?"
result = qa_roberta(question=multi_hop_q, context=complex_text)
print(f"Q: {multi_hop_q}")
print(f"A: {result['answer']} (confidence: {result['score']:.3f})")
print()

# 2. Negation handling
print("2. Negation Handling:")
negation_q = "What were AI systems initially NOT focused on?"
result = qa_roberta(question=negation_q, context=complex_text)
print(f"Q: {negation_q}")
print(f"A: {result['answer']} (confidence: {result['score']:.3f})")
print()

# 3. Comparative questions
print("3. Comparative Questions:")
comparative_q = "How do modern AI systems differ from early AI research?"
result = qa_roberta(question=comparative_q, context=complex_text)
print(f"Q: {comparative_q}")
print(f"A: {result['answer']} (confidence: {result['score']:.3f})")
print()

# 4. Questions with no answer (should return low confidence or empty)
print("4. Unanswerable Questions:")
no_answer_q = "What programming language is best for AI development?"
result = qa_roberta(question=no_answer_q, context=complex_text)
print(f"Q: {no_answer_q}")
print(f"A: {result['answer']} (confidence: {result['score']:.3f})")
if result['score'] < 0.1:
    print("⚠️  Low confidence - question likely unanswerable from context")
print()

# 5. Batch processing multiple questions
print("5. Batch Processing:")
batch_questions = [
    "What enables large language models to capture complex patterns?",
    "Where do recommendation systems work?",
    "What requires careful attention from researchers?"
]

print("Processing multiple questions in batch...")
for q in batch_questions:
    result = qa_distilbert(question=q, context=complex_text)
    print(f"Q: {q}")
    print(f"A: {result['answer'][:100]}..." if len(result['answer']) > 100 else f"A: {result['answer']}")
    print()

## Model Comparison and Performance Tips

### Key Differences Between Models
- **BART vs T5 for Summarization**: BART is pre-trained specifically for text generation tasks, while T5 uses a text-to-text approach
- **DistilBERT vs RoBERTa for QA**: DistilBERT is faster and smaller, RoBERTa typically provides higher accuracy
- **Authentication Benefits**: Authenticated access provides higher rate limits and access to private/gated models

In [ ]:
# Performance Comparison and Optimization Tips
import time

def benchmark_models():
    """Benchmark the performance of different models"""
    print("=== Model Performance Benchmark ===\n")
    
    test_text = complex_text[:500]  # Use first 500 chars for consistent testing
    test_question = "What is artificial intelligence?"
    
    # Summarization benchmark
    print("Summarization Performance:")
    
    # BART timing
    start_time = time.time()
    bart_result = summarizer_bart(test_text, max_length=50, min_length=20)
    bart_time = time.time() - start_time
    print(f"BART: {bart_time:.3f}s - {bart_result[0]['summary_text'][:50]}...")
    
    # T5 timing
    start_time = time.time()
    t5_result = summarizer_t5("summarize: " + test_text, max_length=50, min_length=20)
    t5_time = time.time() - start_time
    print(f"T5: {t5_time:.3f}s - {t5_result[0]['generated_text'][:50]}...")
    
    print(f"\nSummarization Speed Comparison: BART is {t5_time/bart_time:.1f}x {'faster' if bart_time < t5_time else 'slower'} than T5")
    
    print("\n" + "-"*50 + "\n")
    
    # QA benchmark
    print("Question Answering Performance:")
    
    # DistilBERT timing
    start_time = time.time()
    distilbert_result = qa_distilbert(question=test_question, context=test_text)
    distilbert_time = time.time() - start_time
    print(f"DistilBERT: {distilbert_time:.3f}s - {distilbert_result['answer'][:30]}... (conf: {distilbert_result['score']:.3f})")
    
    # RoBERTa timing
    start_time = time.time()
    roberta_result = qa_roberta(question=test_question, context=test_text)
    roberta_time = time.time() - start_time
    print(f"RoBERTa: {roberta_time:.3f}s - {roberta_result['answer'][:30]}... (conf: {roberta_result['score']:.3f})")
    
    print(f"\nQA Speed Comparison: DistilBERT is {roberta_time/distilbert_time:.1f}x {'faster' if distilbert_time < roberta_time else 'slower'} than RoBERTa")

# Run benchmark
benchmark_models()

print("\n" + "="*60)
print("OPTIMIZATION TIPS:")
print("="*60)
print("1. Use GPU when available (device=0) for faster inference")
print("2. Batch multiple inputs together for better throughput")
print("3. Consider model size vs. accuracy trade-offs:")
print("   - DistilBERT: 66M params, fastest")
print("   - BART-large: 400M params, good balance")
print("   - T5-base: 220M params, versatile")
print("4. Cache models locally to avoid re-downloading")
print("5. Use quantization for deployment (not shown here)")
print("6. Adjust max_length parameters based on your needs")

In [ ]:
# Installation Requirements and Setup
print("=== Required Packages ===")
print("To run this notebook, install the following packages:")
print()

requirements = [
    "torch>=1.9.0",
    "transformers>=4.20.0", 
    "huggingface_hub>=0.10.0",
    "tokenizers>=0.12.0",
    "datasets",  # Optional, for additional datasets
    "accelerate",  # For faster training/inference
]

for req in requirements:
    print(f"pip install {req}")

print("\nFor GPU support (optional but recommended):")
print("pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

print("\n" + "="*50)
print("AUTHENTICATION SETUP:")
print("="*50)
print("1. Create account at https://huggingface.co/")
print("2. Generate access token at https://huggingface.co/settings/tokens")
print("3. Set environment variable: export HUGGINGFACE_HUB_TOKEN='your_token'")
print("4. Or use: huggingface-cli login")
print("5. Or use: from huggingface_hub import login; login()")

print("\n✅ Setup complete! You can now run all the cells above.")